### **Gender Estimation using custom Neural network**

### 1. Download dataset
- UTKFace source: https://www.kaggle.com/datasets/jangedoo/utkface-new

In [ ]:
!gdown 1HOiaQcnemJWsYuTEHRpa3jfSFYDex7NB

Downloading...
From: https://drive.google.com/uc?id=1HOiaQcnemJWsYuTEHRpa3jfSFYDex7NB
To: /content/UTKFace.zip
100% 347M/347M [00:01<00:00, 183MB/s]


In [ ]:
!unzip -q UTKFace.zip -d data

In [ ]:
# Number of images in UTKFace
!ls data/UTKFace | wc -l

23708


In [ ]:
!ls -U data/UTKFace | head -5

28_1_4_20170103182238570.jpg.chip.jpg
37_1_0_20170117203545246.jpg.chip.jpg
35_0_0_20170117170113915.jpg.chip.jpg
45_0_0_20170117182743885.jpg.chip.jpg
37_0_0_20170116200540241.jpg.chip.jpg


### 2. Import libraries

In [ ]:
from google.colab import files
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime

### 3. Process dataset

*  Reading the image files as 3D NumPy arrays. Note, we'll use 3-channeled RGB images for training the model, so each array will have a shape of `[ img_width , img_height , 3 ]`.

*  Split the filename so as to parse the gender of the person in corresponding image. We use the `tf.strings.split()` method for performing this task.

*  We one-hot encode the gender, as we'll perform *a two-class* classification.

Once this operations have been performed, we are left with $N$ samples where each sample consists of image array `[ 128 , 128 , 3 ]` and its corresponding label ( one-hot encoded ), the gender of that person, which has a shape `[ 1 , 2 ]`

We'll use `tf.data.Dataset` as it helps us to process the data faster, taking advantage of parallel computing. The above two operations will be mapped on each filename using `tf.data.Dataset.map` method.

In [8]:
# Image size for our model.
MODEL_INPUT_IMAGE_SIZE = [128, 128]

# Fraction of the dataset to be split test set
TRAIN_TEST_SPLIT = 0.3

# Trick to one-hot encode the label.
y1 = tf.constant([1., 0.], dtype='float32') # Male
y2 = tf.constant([0., 1.], dtype='float32') # Female

In [9]:
def parse_image(filename):
    # Read the image from the filename and resize it.
    image_raw = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_raw, channels=3) 
    image = tf.image.resize(image, MODEL_INPUT_IMAGE_SIZE) / 255

    # Split the filename to get the age and the gender. Convert the age ( str ) and the gender ( str ) to dtype float32.
    parts = tf.strings.split(tf.strings.split(filename, '/')[2], '_')

    # One-hot encode the label
    gender = tf.strings.to_number(parts[1])
    gender_onehot = (gender * y2) + ((1 - gender) * y1)

    return image, gender_onehot

In [10]:
# List all the image files in the given directory.
list_ds = tf.data.Dataset.list_files('data/UTKFace/*', shuffle=True)
# Map `parse_image` method to all filenames.
dataset = list_ds.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)


We create two splits from our dataset, one for training the model and another for testing the model. The fraction of the dataset which will be used for testing the model is determined by `TRAIN_TEST_SPLIT`.


In [11]:
# Create train and test splits of the dataset.
num_examples_in_test_ds = int(dataset.cardinality().numpy() * TRAIN_TEST_SPLIT)

test_ds = dataset.take(num_examples_in_test_ds)
train_ds = dataset.skip(num_examples_in_test_ds)

print('Num examples in train ds {}'.format( train_ds.cardinality()))
print('Num examples in test ds {}'.format( test_ds.cardinality()))

Num examples in train ds 16596
Num examples in test ds 7112



### 4. The CNN Model

Our aim is to develop a model which has lesser parameters ( which implies lesser inference time and size ) but powerful enough so that it can generalize better.

* The model takes in a batch of shape `[ None , 128 , 128 , 3 ]` and performs a number of convolutions on it as determined by `num_blocks`.
* Each block consists of a sequence of layers : `Conv2D -> BatchNorm -> LeakyReLU`

* If `lite_model` is set to `True`, we use  Separable Convolutionswhich have lesser parameters. We could achieve a *faster* model, compromising its performance.

* We stack such`num_blocks` blocks sequentially, where the no. of filters for each layer is taken from `num_filters`.

* Next we add a number of `Dense` layers to learn the features extracted by convolutional layers. Note, we also add a `Dropout` layer, to reduce overfitting. The `rate` for each `Dropout` layer is decreased subsequently for each layer, so that the learnability of `Dense` layer with lesser units.

* The last `Dense` layer applies the softmax activation function which yields a probability distribution for the two classes `male` and `female`.

* The output of the model is a tensor with shape `[ None, 2 ]`


In [12]:
# Negative slope coefficient for LeakyReLU.
leaky_relu_alpha = 0.2
lite_model = True

In [13]:
# Define the conv block.
def conv(x, num_filters, kernel_size=(3, 3), strides=1):
    if lite_model:
        x = tf.keras.layers.SeparableConv2D(num_filters,
                                            kernel_size=kernel_size,
                                            strides=strides, 
                                            use_bias=False,
                                            kernel_initializer=tf.keras.initializers.HeNormal(),
                                            kernel_regularizer=tf.keras.regularizers.L2(1e-5)
                                           )(x)
    else:
        x = tf.keras.layers.Conv2D(num_filters,
                                   kernel_size=kernel_size,
                                   strides=strides,
                                   use_bias=False,
                                   kernel_initializer=tf.keras.initializers.HeNormal(),
                                   kernel_regularizer=tf.keras.regularizers.L2(1e-5)
                                  )(x)

    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(leaky_relu_alpha)(x)
    return x

In [15]:
def dense(x, filters, dropout_rate):
    x = tf.keras.layers.Dense(filters, kernel_regularizer=tf.keras.regularizers.L2(0.1), 
                              bias_regularizer=tf.keras.regularizers.L2(0.1))(x)
    x = tf.keras.layers.LeakyReLU(alpha=leaky_relu_alpha)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    return x

In [16]:
# No. of convolution layers to be added.
num_blocks = 5
# Num filters for each conv layer.
num_filters = [16, 32, 64, 128, 256, 256]
# Kernel sizes for each conv layer.
kernel_sizes = [3, 3, 3, 3, 3, 3]

# Init a Input Layer.
inputs = tf.keras.layers.Input(shape=MODEL_INPUT_IMAGE_SIZE + [3])

# Add conv blocks sequentially
x = inputs
for i in range(num_blocks):
    x = conv(x, num_filters=num_filters[i], kernel_size=kernel_sizes[i])
    x = tf.keras.layers.MaxPooling2D()(x)

# Flatten the output of the last Conv layer.
x = tf.keras.layers.Flatten()(x)
conv_output = x 

# Add Dense layers ( Dense -> LeakyReLU -> Dropout )
x = dense(conv_output, 256, 0.6)
x = dense(x, 64, 0.4)
x = dense(x, 32, 0.2)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

# Build the Model
model = tf.keras.models.Model(inputs, outputs)

# Uncomment the below to view the summary of the model.
model.summary()
# tf.keras.utils.plot_model( model , to_file='architecture.png' )

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 separable_conv2d (Separable  (None, 126, 126, 16)     75        
 Conv2D)                                                         
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 16)     64        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 126, 126, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 16)       0         
 )                                                               
                                                             

### 4. Compiling the model

Once we've defined the architecture for our model, we'll compile our Keras model and also initialize some useful callbacks.

* As we're performing classification, we'll use the Categorical Crossentropy loss function. See [`tf.keras.losses.CategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy).

* We'll use the Adam optimizer for training our model. See [`tf.keras.optimizers.Adam`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).

* For evaluating the performance of our model, we measure the accuracy of our model. See [`tf.keras.metrics.Accuracy`](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Accuracy).


#### Callbacks:

* [`tf.keras.callbacks.ModelCheckpoint`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) to save the Keras model as an H5 file after every epoch.

* [`tf.keras.callbacks.TensorBoard`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) to visualize the training with TensorBoard.

* [`tf.keras.callbacks.EarlyStopping`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) to stop the training when the evaluation metric i.e the MAE stops improving on the test dataset.

In [17]:
learning_rate = 0.0001
num_epochs = 10 
batch_size = 128

train_ds = train_ds.batch(batch_size).repeat(num_epochs)
test_ds = test_ds.batch(batch_size).repeat(num_epochs)

save_dir = 'train-1/cp.ckpt'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(save_dir)

logdir = os.path.join("tb_logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

model.compile( 
    loss=tf.keras.losses.categorical_crossentropy , 
    optimizer=tf.keras.optimizers.Adam(learning_rate) , 
    metrics=['accuracy']
)

### 5. Train and Evaluate the Model

Start the training loop with all callbacks packed in.


In [18]:
model.fit( 
    train_ds, 
    epochs=num_epochs,  
    validation_data=test_ds
)

Epoch 1/10
1300/1300 [==============================] - 387s 285ms/step - loss: 14.0444 - accuracy: 0.7702 - val_loss: 1.7864 - val_accuracy: 0.8625
Epoch 2/10
1300/1300 [==============================] - 371s 283ms/step - loss: 0.9205 - accuracy: 0.8650 - val_loss: 0.4916 - val_accuracy: 0.8899
Epoch 3/10
1300/1300 [==============================] - 370s 282ms/step - loss: 0.4312 - accuracy: 0.8872 - val_loss: 0.3600 - val_accuracy: 0.9058
Epoch 4/10
1300/1300 [==============================] - 371s 283ms/step - loss: 0.3600 - accuracy: 0.9030 - val_loss: 0.3177 - val_accuracy: 0.9171
Epoch 5/10
1300/1300 [==============================] - 374s 285ms/step - loss: 0.3251 - accuracy: 0.9148 - val_loss: 0.3001 - val_accuracy: 0.9212
Epoch 6/10
1300/1300 [==============================] - 374s 285ms/step - loss: 0.2944 - accuracy: 0.9273 - val_loss: 0.2632 - val_accuracy: 0.9421
Epoch 7/10
1300/1300 [==============================] - 380s 290ms/step - loss: 0.2720 - accuracy: 0.9364 - val

Evaluate the Model.

In [19]:
p = model.evaluate(test_ds)
print('loss is {} \n accuracy is {} %'.format(p[0], p[1] * 100))

560/560 [==============================] - 54s 96ms/step - loss: 0.1861 - accuracy: 0.9746
loss is 0.1860535591840744 
 accuracy is 97.45782017707825 %



Save the Keras model to the local disk, so that we can resume training if needed.



In [20]:
model_name = 'model_gender' #@param {type: "string"}
model_name_ = model_name + '.h5'

model.save(model_name_)
files.download(model_name_) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 ### 6. Convert to TensorFlow Lite format

Our model is to be deployed in an Android app, where we'll use [TF Lite Android](https://bintray.com/google/tensorflow/tensorflow-lite) package to parse the model and make predictions.

We use the `TFLiteConverter` API to convert our Keras Model ( `.h5` ) to a TF Lite buffer ( `.tflite` ). We'll produce two TF Lite buffers, one with float16 quantization and other non-quantized model.


In [21]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
buffer = converter.convert()

open('{}_q.tflite'.format(model_name), 'wb').write(buffer)
files.download('{}_q.tflite'.format(model_name))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


For conversion to a non-quantized TF Lite buffer.


In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
buffer = converter.convert()

open('{}_nonq.tflite'.format(model_name), 'wb').write(buffer)
files.download('{}_nonq.tflite'.format(model_name))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>